# 基于word2vec的词嵌入

在不深入细节的情况下，笔者将解释如何创建语句向量（Sentence Vectors），以及如何基于它们在其上创建机器学习模型。鄙人是GloVe向量，word2vec和fasttext的粉丝（但平时还是用word2vec较多）。在这篇文章中，笔者使用的文本分类模型是基于Word2vec词向量模型（100维）。


训练word2vec词向量:

In [7]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
import re
import jieba

In [2]:
# 数据整合，输出新数据
def merge_file(file1,file2):
    feature = pd.read_csv(file1,sep=",")
    label = pd.read_csv(file2,sep=",")
    data = feature.merge(label,on='id') # 按照id列整合两个训练集表
    data["X"] = data[["title","content"]].apply(lambda x:"".join([str(x[0]),str(x[1])]),axis=1) # "title","content"两列合为一列"X"
    df = data[['id', 'title', 'content', 'label', 'X']]
    return df


data = merge_file("Train_DataSet.csv","Train_DataSet_Label.csv")

In [3]:
# 停用词加载和清洗
def stopwords_list(filepath):
    stopwords=[line.strip() for line in open(filepath,'r',encoding='utf-8').readlines()]
    return stopwords

stopwords=stopwords_list('stopwords.txt')

def stop_clean(strs):
    strs = [w for w in strs if not w in stopwords]
    return strs

# 只保留汉字
def character_save(strs):
    strs = re.findall(r'[\u4e00-\u9fa5]',strs)
    return strs

In [8]:
#处理训练集，将训练集的文本信息和label信息合并，清洗特殊符合，同时将文本内容进行分词
def data_pre_handle(df):
    # 1.去空值
    dataDropNa=df.dropna(axis=0, how='any')  
#    dfDropNa["X"]=dataDropNa["X"].apply(lambda x: str(x).replace("\\n","").replace(".","").replace("\n","").replace("　","").replace("↓","").replace("/","").replace("|","").replace(" ",""))
    dataDropNa["X_split"]=dataDropNa["X"]
    dataDropNa["X_split"]=dataDropNa["X_split"].apply(lambda x:"".join(character_save(x)))
    dataDropNa["X_split"]=dataDropNa["X_split"].apply(lambda x:" ".join(jieba.cut(x)))
    dataDropNa["X_split"]=dataDropNa["X_split"].apply(lambda x:"".join(stop_clean(x)))
    # 程序运行慢是因为stopwords太大

    return dataDropNa
 
data = data_pre_handle(data)

C:\Users\dongy\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\dongy\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\dongy\AppData\Local\Temp\jieba.cache
Loading model cost 0.630 seconds.
Prefix dict has been built succesfully.
C:\Users\dongy\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:8: SettingW

In [9]:
#处理测试数据
def process_test(test_name):
    test=pd.read_csv(test_name,sep=",")
    test["X"]=test[["title","content"]].apply(lambda x:"".join([str(x[0]),str(x[1])]),axis=1)
    test["X_split"]=test["X"].apply(lambda x:"".join(character_save(x)))
    test["X_split"]=test["X_split"].apply(lambda x:" ".join(jieba.cut(x)))
    test["X_split"]=test["X_split"].apply(lambda x:"".join(stop_clean(x)))
    
    return test
 
testData=process_test("Test_DataSet.csv")

In [213]:
# import jieba
# #处理训练集，将训练集的文本信息和label信息合并，清洗特殊符号，同时将文本内容进行分词
# def merge_feature_label(feature_name,label_name):
#     feature=pd.read_csv(feature_name,sep=",")
#     label=pd.read_csv(label_name,sep=",")
#     data=feature.merge(label,on='id')
#     data["X"]=data[["title","content"]].apply(lambda x:"".join([str(x[0]),str(x[1])]),axis=1)
#     # 丢掉缺失值
#     dataDropNa=data.dropna(axis=0, how='any')
#     print(dataDropNa.info())
#     dataDropNa["X"]=dataDropNa["X"].apply(lambda x: str(x).replace("\\n","").replace(".","").replace("\n","").replace("　","").replace("↓","").replace("/","").replace("|","").replace(" ",""))
#     dataDropNa["X_split"]=dataDropNa["X"].apply(lambda x:" ".join(jieba.cut(x)))
#     return dataDropNa
 


In [214]:
# #处理测试数据
# def process_test(test_name):
#     test=pd.read_csv(test_name,sep=",")
#     # apply()方法能劫持另外一个对象的方法，继承另外一个对象的属性 
#     # lambda https://blog.csdn.net/zjuxsl/article/details/79437563
#     #  join() 方法用于将序列中的元素以指定的字符连接生成一个新的字符串。
#     test["X"]=test[["title","content"]].apply(lambda x:"".join([str(x[0]),str(x[1])]),axis=1)
#     # 列名信息
#     print(test.info())
#     # replace(old, new) 方法把字符串中的 old（旧字符串） 替换成 new(新字符串)，如果指定第三个参数max，则替换不超过 max 次。
#     test["X"]=test["X"].apply(lambda x: str(x).replace("\\n","").replace(".","").replace("\n","").replace("　","").replace("↓","").replace("/","").replace("|","").replace(" ",""))
#     # jieba.cut()按照词性切分“我爱中国”切分成我爱中国
#     test["X_split"]=test["X"].apply(lambda x:" ".join(jieba.cut(x)))
#     return test
 


In [10]:
# dataDropNa=merge_feature_label("Train_DataSet.csv","Train_DataSet_Label.csv")
dataDropNa = data
data_reindex = dataDropNa.reset_index(drop=True)
# data_reindex

In [216]:
# testData=process_test("Test_DataSet.csv")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7356 entries, 0 to 7355
Data columns (total 4 columns):
id         7356 non-null object
title      7356 non-null object
content    7288 non-null object
X          7356 non-null object
dtypes: object(4)
memory usage: 230.0+ KB
None


In [11]:
datatest = testData

In [12]:
def data_to_list(df):
    datas = []
    for content in df["X_split"][0:]:
        dataClean = content.split(" ")
        datas.append(dataClean)
    
    return datas   

In [13]:
X = data_to_list(data_reindex) 

In [14]:
len(X)

7265

In [15]:
Y = data_to_list(datatest) 

In [16]:
len(Y)

7356

In [17]:
import gensim
model = gensim.models.Word2Vec(X, size=100) # 去掉, iter=10

X是经分词后的文本构成的list，也就是tokens的列表的列表.

    注意，Word2Vec还有3个值得关注的参数，iter是模型训练时迭代的次数，假
    如参与训练的文本量较少，就需要把这个参数调大一些；sg是模型训练算法的
    类别，1 代表 skip-gram，;0代表 CBOW;window控制窗口，它指当前词和预测
    词之间的最大距离，如果设得较小，那么模型学习到的是词汇间的功能性特征
    （词性相异），如果设置得较大，会学习到词汇之间的相似性特征（词性相
    同）的大小，假如语料够多，笔者一般会设置得大一些，8~10。

In [224]:
embeddings_index = dict(zip(model.wv.index2word, model.wv.syn0))

print('Found %s word vectors.' % len(embeddings_index))



C:\Users\dongy\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  if __name__ == '__main__':


Found 52617 word vectors.


In [227]:
embeddings_index

{'供认不讳': array([-0.07187846,  0.46787694,  0.5030147 , -0.32405788,  0.02490417,
         0.6072453 , -0.38775653, -0.160134  ,  0.4134022 ,  0.6077835 ,
         0.47358188, -0.07065018, -0.0486949 , -0.9441076 ,  0.10145769,
         0.566353  ,  1.0950868 ,  0.41233987,  0.44793966,  0.10132284,
         0.93949246,  0.2889971 ,  0.17973733, -0.3538721 , -0.23824193,
        -1.1458646 , -0.34242442,  0.88796216,  0.52198225, -0.14899893,
        -0.11428948, -0.73426205, -1.3511981 , -0.17775635, -0.30511034,
         0.6132094 ,  0.5825379 ,  0.6792376 , -0.46993423, -0.02395689,
        -0.3007815 ,  0.13329075,  0.10809273,  0.14609274, -0.0419881 ,
         0.6993209 ,  1.323244  , -0.2847326 , -1.2116897 ,  0.12409366,
        -0.2724035 ,  0.7859957 , -0.78216934, -0.04174868, -0.13323404,
        -0.09802432,  0.9642111 , -1.444319  ,  0.1700998 ,  0.4728428 ,
        -1.0553417 , -0.26814803,  0.31279424,  0.7908785 , -0.2013801 ,
         0.58100355,  0.745722  ,  0.086533

该函数会将语句转化为一个标准化的向量（Normalized Vector）

In [228]:
def sent2vec(s):
    words = str(s)
    words = word_tokenize(words)
#     words = [w for w in words if not w in stop_words]
    words = [w for w in words]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

In [229]:
x = data_reindex.X_split.values 
y = data_reindex.label.values

In [230]:
xtrain, xvalid, ytrain, yvalid = train_test_split(x, y,
                                                  stratify=y, 
                                                  random_state=42,
                                                  test_size=0.1, shuffle=True)
# xtrain相当于xtrain_total  xvalid相当于xtest_total ytrain相当于ytrain_total

In [231]:
xvalid.shape

(727,)

对训练集和验证集使用上述函数，进行文本向量化处理

In [232]:
xtrain_w2v  = [sent2vec(x) for x in tqdm(xtrain)]
xvalid_w2v  = [sent2vec(x) for x in tqdm(xvalid)]



100%|███████████████████████████████████████████████████████████████████████████████| 727/727 [00:01<00:00, 550.75it/s]


In [233]:
xvalid_w2v

[array([-0.03997131,  0.06358811, -0.09772618,  0.07020188, -0.09666847,
         0.04914194, -0.00451302, -0.05297343,  0.08936652,  0.03840062,
        -0.05145815,  0.03173135, -0.05399966, -0.25563672,  0.13459952,
         0.1804107 ,  0.08845335,  0.156936  ,  0.01726459, -0.22377987,
         0.18500055, -0.01417801,  0.03666887, -0.10803259,  0.0255877 ,
        -0.0013028 ,  0.01200979,  0.03267233, -0.18179812, -0.09884471,
         0.05125026, -0.19143587, -0.07764337, -0.22028613,  0.01239196,
         0.04105956,  0.01063027,  0.10927883,  0.00084626, -0.0661239 ,
        -0.05113741, -0.00481801, -0.02126065, -0.09424618,  0.14641108,
         0.10403063,  0.08976985,  0.01861598, -0.06050608,  0.07117131,
        -0.00952166,  0.10212105, -0.07901952, -0.04518557, -0.03311612,
         0.14717843,  0.08582355,  0.02940776, -0.05721264,  0.01822278,
        -0.13832961, -0.09008108,  0.07060866,  0.15575574, -0.01515808,
         0.01499247,  0.06710843,  0.04059539,  0.1

In [234]:
len(xvalid_w2v[0])


100

In [235]:
xtrain_w2v  = np.array(xtrain_w2v)
xvalid_w2v  = np.array(xvalid_w2v)

In [237]:
(xtrain_w2v).shape

(6538, 100)

In [239]:
xvalid_w2v.shape


(727, 100)

In [161]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """对数损失度量（Logarithmic Loss  Metric）的多分类版本。
    :param actual: 包含actual target classes的数组
    :param predicted: 分类预测结果矩阵, 每个类别都有一个概率
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [164]:
xvalid_w2v.shape

(727, 100)

让我们看看xgboost在Glove词向量特征的表现如何：

基于word2vec特征使用XGB文本分类器：

In [162]:
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1, silent=False)
clf.fit(xtrain_w2v, ytrain)
predictions = clf.predict_proba(xvalid_w2v)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.606 


In [ ]:
preds=predictions
preds=np.argmax(preds,axis=1)
test_pred=pd.DataFrame(preds)
test_pred.columns=["label"]
print(test_pred.shape)
test_pred["id"]=list(testData["id"])
test_pred[["id","label"]].to_csv('0000000000.csv',index=None)

In [ ]:
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1, silent=False)
clf.fit(x_test_w2v, ytrain)
predictions = clf.predict_proba(x_test_w2v)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

我们可以看到，简单的对参数进行微调，就提高基于word2vec词向量特征的
xgboost得分！ 相信我，你还可以从中继续“压榨”出更优秀的表现！


## 深度学习（Deep Learning）

这是一个深度学习大行其道的时代！ 文本分类问题在它的指引下得到了突飞猛进
的发展！ 在这里，我们将在word2vec功能上训练LSTM和简单的全连接网络（Dense
Network）。
让我们先从全连接网络开始：

在使用神经网络前，对数据进行缩放：

In [64]:
scl = preprocessing.StandardScaler()
xtrain_w2v_scl = scl.fit_transform(xtrain_w2v)
xvalid_w2v_scl = scl.transform(xvalid_w2v)

对标签进行binarize处理

In [65]:
# xtrain_v_scl = np_utils.to_categorical(xtrain)
# xvalid_v_scl = np_utils.to_categorical(xvalid)


ytrain_enc = np_utils.to_categorical(ytrain)
yvalid_enc = np_utils.to_categorical(yvalid)

创建1个3层的序列神经网络（Sequential Neural Net）
        




In [77]:
model = Sequential()
# 300 改  100
model.add(Dense(300, input_dim=100, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
# 300 改  100
model.add(Dense(300, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
# 14 改  3
model.add(Dense(3))
model.add(Activation('softmax'))

对模型进行编译和拟合：

In [79]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(xtrain_w2v_scl, y=ytrain_enc, batch_size=64, 
          epochs=30, verbose=1, 
          validation_data=(xvalid_w2v_scl, yvalid_enc))

Train on 6538 samples, validate on 727 samples
Epoch 1/30
6538/6538 [==============================] - ETA: 33s - loss: 0.18 - ETA: 3s - loss: 0.2542 - ETA: 1s - loss: 0.276 - ETA: 1s - loss: 0.295 - ETA: 0s - loss: 0.293 - ETA: 0s - loss: 0.290 - ETA: 0s - loss: 0.293 - ETA: 0s - loss: 0.299 - ETA: 0s - loss: 0.298 - ETA: 0s - loss: 0.296 - ETA: 0s - loss: 0.295 - 1s 151us/step - loss: 0.2949 - val_loss: 0.6979
Epoch 2/30
6538/6538 [==============================] - ETA: 0s - loss: 0.213 - ETA: 0s - loss: 0.277 - ETA: 0s - loss: 0.312 - ETA: 0s - loss: 0.300 - ETA: 0s - loss: 0.289 - ETA: 0s - loss: 0.293 - ETA: 0s - loss: 0.283 - ETA: 0s - loss: 0.285 - ETA: 0s - loss: 0.285 - ETA: 0s - loss: 0.285 - 1s 92us/step - loss: 0.2866 - val_loss: 0.7381
Epoch 3/30
6538/6538 [==============================] - ETA: 0s - loss: 0.425 - ETA: 0s - loss: 0.282 - ETA: 0s - loss: 0.277 - ETA: 0s - loss: 0.270 - ETA: 0s - loss: 0.273 - ETA: 0s - loss: 0.277 - ETA: 0s - loss: 0.279 - ETA: 0s - loss: 0

你需要不断的对神经网络的参数进行调优，添加更多层，增加Dropout以获得更好
的结果。 在这里，笔者只是简单的实现下，追求速度而不是最终效果，并且它比
没有任何优化的xgboost取得了更好的结果:)

为了更进一步，笔者使用LSTM，我们需要对文本数据进行Tokenize：

In [80]:
token = text.Tokenizer(num_words=None)
max_len = 70

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

对文本序列进行zero填充:

In [81]:
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

基于已有的数据集中的词汇创建一个词嵌入矩阵（Embedding Matrix）:

In [82]:
embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|█████████████████████████████████████████████████████████████████████| 188819/188819 [00:00<00:00, 1014177.73it/s]


基于前面训练的Word2vec词向量，使用1个两层的LSTM模型:

In [75]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     100,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, verbose=1, validation_data=(xvalid_pad, yvalid_enc))

Train on 6538 samples, validate on 727 samples
Epoch 1/100
6538/6538 [==============================] - ETA: 7s - loss: 1.104 - ETA: 5s - loss: 1.058 - ETA: 3s - loss: 1.036 - ETA: 3s - loss: 1.025 - ETA: 2s - loss: 1.023 - ETA: 2s - loss: 1.020 - ETA: 2s - loss: 1.017 - ETA: 1s - loss: 1.008 - ETA: 1s - loss: 1.019 - ETA: 0s - loss: 1.009 - ETA: 0s - loss: 1.006 - ETA: 0s - loss: 1.002 - 4s 676us/step - loss: 0.9990 - val_loss: 0.9371
Epoch 2/100
6538/6538 [==============================] - ETA: 3s - loss: 0.940 - ETA: 3s - loss: 0.934 - ETA: 2s - loss: 0.951 - ETA: 2s - loss: 0.954 - ETA: 2s - loss: 0.957 - ETA: 1s - loss: 0.956 - ETA: 1s - loss: 0.957 - ETA: 1s - loss: 0.958 - ETA: 1s - loss: 0.957 - ETA: 0s - loss: 0.951 - ETA: 0s - loss: 0.952 - ETA: 0s - loss: 0.954 - 4s 584us/step - loss: 0.9546 - val_loss: 0.9206
Epoch 3/100
6538/6538 [==============================] - ETA: 3s - loss: 0.963 - ETA: 2s - loss: 0.945 - ETA: 2s - loss: 0.949 - ETA: 2s - loss: 0.947 - ETA: 2s - loss

6538/6538 [==============================] - ETA: 5s - loss: 0.680 - ETA: 5s - loss: 0.693 - ETA: 4s - loss: 0.691 - ETA: 3s - loss: 0.706 - ETA: 3s - loss: 0.697 - ETA: 3s - loss: 0.708 - ETA: 2s - loss: 0.713 - ETA: 2s - loss: 0.713 - ETA: 1s - loss: 0.720 - ETA: 1s - loss: 0.729 - ETA: 0s - loss: 0.727 - ETA: 0s - loss: 0.729 - 6s 935us/step - loss: 0.7312 - val_loss: 0.6938
Epoch 42/100
6538/6538 [==============================] - ETA: 5s - loss: 0.697 - ETA: 5s - loss: 0.718 - ETA: 4s - loss: 0.717 - ETA: 4s - loss: 0.719 - ETA: 3s - loss: 0.719 - ETA: 3s - loss: 0.716 - ETA: 2s - loss: 0.711 - ETA: 2s - loss: 0.713 - ETA: 1s - loss: 0.721 - ETA: 1s - loss: 0.722 - ETA: 0s - loss: 0.721 - ETA: 0s - loss: 0.719 - 6s 954us/step - loss: 0.7225 - val_loss: 0.6872
Epoch 43/100
6538/6538 [==============================] - ETA: 5s - loss: 0.745 - ETA: 4s - loss: 0.724 - ETA: 4s - loss: 0.735 - ETA: 3s - loss: 0.728 - ETA: 3s - loss: 0.727 - ETA: 3s - loss: 0.726 - ETA: 2s - loss: 0.724 -

6538/6538 [==============================] - ETA: 5s - loss: 0.587 - ETA: 4s - loss: 0.618 - ETA: 4s - loss: 0.629 - ETA: 4s - loss: 0.629 - ETA: 3s - loss: 0.621 - ETA: 3s - loss: 0.620 - ETA: 2s - loss: 0.622 - ETA: 2s - loss: 0.624 - ETA: 1s - loss: 0.624 - ETA: 1s - loss: 0.621 - ETA: 0s - loss: 0.622 - ETA: 0s - loss: 0.625 - 6s 961us/step - loss: 0.6262 - val_loss: 0.6945
Epoch 82/100
6538/6538 [==============================] - ETA: 5s - loss: 0.579 - ETA: 4s - loss: 0.613 - ETA: 4s - loss: 0.627 - ETA: 4s - loss: 0.637 - ETA: 3s - loss: 0.628 - ETA: 3s - loss: 0.632 - ETA: 2s - loss: 0.626 - ETA: 2s - loss: 0.624 - ETA: 1s - loss: 0.628 - ETA: 1s - loss: 0.626 - ETA: 0s - loss: 0.629 - ETA: 0s - loss: 0.631 - 6s 932us/step - loss: 0.6322 - val_loss: 0.6906
Epoch 83/100
6538/6538 [==============================] - ETA: 5s - loss: 0.618 - ETA: 5s - loss: 0.642 - ETA: 4s - loss: 0.642 - ETA: 4s - loss: 0.637 - ETA: 3s - loss: 0.636 - ETA: 3s - loss: 0.638 - ETA: 2s - loss: 0.640 -

现在，我们看到分数小于0.5。 我跑了很多个epochs都没有获得最优的结果，但我
们可以使用early stopping来停止在最佳的迭代节点。

那我们该如何使用early stopping？

好吧，其实很简单的。 让我们再次compile模型：


In [83]:
#基于前面训练的Word2vec词向量，使用1个两层的LSTM模型
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     100,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

在模型拟合时，使用early stopping这个回调函数（Callback Function）

In [84]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

Train on 6538 samples, validate on 727 samples
Epoch 1/100
6538/6538 [==============================] - ETA: 12s - loss: 1.11 - ETA: 10s - loss: 1.07 - ETA: 8s - loss: 1.0431 - ETA: 7s - loss: 1.034 - ETA: 6s - loss: 1.016 - ETA: 5s - loss: 1.012 - ETA: 4s - loss: 1.018 - ETA: 3s - loss: 1.016 - ETA: 3s - loss: 1.020 - ETA: 2s - loss: 1.013 - ETA: 1s - loss: 1.011 - ETA: 0s - loss: 1.011 - 11s 2ms/step - loss: 1.0072 - val_loss: 0.9347
Epoch 2/100
6538/6538 [==============================] - ETA: 8s - loss: 0.950 - ETA: 8s - loss: 0.942 - ETA: 7s - loss: 0.951 - ETA: 6s - loss: 0.961 - ETA: 5s - loss: 0.965 - ETA: 5s - loss: 0.966 - ETA: 4s - loss: 0.964 - ETA: 3s - loss: 0.960 - ETA: 2s - loss: 0.958 - ETA: 2s - loss: 0.952 - ETA: 1s - loss: 0.953 - ETA: 0s - loss: 0.954 - 11s 2ms/step - loss: 0.9545 - val_loss: 0.9138
Epoch 3/100
6538/6538 [==============================] - ETA: 9s - loss: 0.924 - ETA: 8s - loss: 0.932 - ETA: 7s - loss: 0.942 - ETA: 6s - loss: 0.937 - ETA: 6s - loss:

一个可能的问题是：为什么我会使用这么多的dropout？ 嗯，fit模型时，没有或
很少的dropout，你会出现过拟合（Overfit）:)

让我们看看双向长短时记忆(Bi-Directional LSTM)是否可以给我们带来更好的结
果。 对于Keras来说，使用Bilstm小菜一碟:)

基于前面训练的Word2vec词向量，构建1个2层的Bidirectional LSTM :

In [87]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     100,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(100, dropout=0.3, recurrent_dropout=0.3)))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

在模型拟合时，使用early stopping这个回调函数（Callback Function）

In [88]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

Train on 6538 samples, validate on 727 samples
Epoch 1/100
6538/6538 [==============================] - ETA: 37s - loss: 1.18 - ETA: 31s - loss: 1.10 - ETA: 27s - loss: 1.06 - ETA: 24s - loss: 1.05 - ETA: 21s - loss: 1.04 - ETA: 18s - loss: 1.04 - ETA: 15s - loss: 1.04 - ETA: 13s - loss: 1.04 - ETA: 10s - loss: 1.04 - ETA: 7s - loss: 1.0342 - ETA: 4s - loss: 1.029 - ETA: 2s - loss: 1.025 - 37s 6ms/step - loss: 1.0225 - val_loss: 0.9310
Epoch 2/100
6538/6538 [==============================] - ETA: 30s - loss: 0.99 - ETA: 27s - loss: 0.96 - ETA: 25s - loss: 0.96 - ETA: 22s - loss: 0.96 - ETA: 20s - loss: 0.96 - ETA: 17s - loss: 0.96 - ETA: 14s - loss: 0.95 - ETA: 12s - loss: 0.95 - ETA: 9s - loss: 0.9592 - ETA: 7s - loss: 0.959 - ETA: 4s - loss: 0.960 - ETA: 2s - loss: 0.959 - 35s 5ms/step - loss: 0.9550 - val_loss: 0.8945
Epoch 3/100
6538/6538 [==============================] - ETA: 29s - loss: 0.93 - ETA: 27s - loss: 0.92 - ETA: 24s - loss: 0.92 - ETA: 22s - loss: 0.91 - ETA: 19s - los

很接近最优结果了！ 让我们尝试两层的GRU：

In [91]:
# 基于前面训练的Word2vec词向量，构建1个2层的GRU模型
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     100,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(GRU(100, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
model.add(GRU(100, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

模型拟合时，使用early stopping这个回调函数（Callback Function）

In [92]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

Train on 6538 samples, validate on 727 samples
Epoch 1/100
6538/6538 [==============================] - ETA: 1:26 - loss: 191550912.000 - ETA: 1:16 - loss: 144895252.000 - ETA: 1:09 - loss: 126196509.333 - ETA: 1:01 - loss: 114855370.000 - ETA: 54s - loss: 104843387.200 - ETA: 47s - loss: 97841428.0000 - ETA: 40s - loss: 92555896.000 - ETA: 33s - loss: 85884000.500 - ETA: 26s - loss: 81184875.111 - ETA: 19s - loss: 76364344.600 - ETA: 12s - loss: 71684774.000 - ETA: 5s - loss: 67543961.500 - 94s 14ms/step - loss: 65012394.1377 - val_loss: 1.0640
Epoch 2/100
6538/6538 [==============================] - ETA: 1:18 - loss: 14105682.00 - ETA: 1:13 - loss: 11961565.00 - ETA: 1:06 - loss: 13269673.33 - ETA: 1:00 - loss: 12981357.75 - ETA: 53s - loss: 11885111.6000 - ETA: 46s - loss: 12029065.333 - ETA: 39s - loss: 12294347.428 - ETA: 32s - loss: 12526324.375 - ETA: 25s - loss: 11878822.722 - ETA: 19s - loss: 11518433.600 - ETA: 12s - loss: 11463350.909 - ETA: 5s - loss: 11814845.416 - 92s 14m

Epoch 18/100
6538/6538 [==============================] - ETA: 1:26 - loss: 74643.757 - ETA: 1:18 - loss: 60087.470 - ETA: 1:11 - loss: 51907.558 - ETA: 1:03 - loss: 123613.73 - ETA: 56s - loss: 110973.4203 - ETA: 49s - loss: 227642.881 - ETA: 42s - loss: 314028.148 - ETA: 35s - loss: 360501.801 - ETA: 27s - loss: 332244.162 - ETA: 20s - loss: 363329.483 - ETA: 13s - loss: 358185.024 - ETA: 5s - loss: 330063.198 - 97s 15ms/step - loss: 339206.4700 - val_loss: 1.0024


太好了！ 比我们以前的模型好多了！ 持续优化，模型的性能将不断提高。



In [ ]:
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

In [27]:
a = list(dataDropNa["X_split"][0])

In [20]:
a = "他来到了网易杭研大厦"
seg_list = list(jieba.cut(a))


In [5]:
#处理测试数据
def process_test(test_name):
    test=pd.read_csv(test_name,sep=",")
    # apply()方法能劫持另外一个对象的方法，继承另外一个对象的属性 
    # lambda https://blog.csdn.net/zjuxsl/article/details/79437563
    #  join() 方法用于将序列中的元素以指定的字符连接生成一个新的字符串。
    test["X"]=test[["title","content"]].apply(lambda x:"".join([str(x[0]),str(x[1])]),axis=1)
    # 列名信息
    print(test.info())
    # replace(old, new) 方法把字符串中的 old（旧字符串） 替换成 new(新字符串)，如果指定第三个参数max，则替换不超过 max 次。
    test["X"]=test["X"].apply(lambda x: str(x).replace("\\n","").replace(".","").replace("\n","").replace("　","").replace("↓","").replace("/","").replace("|","").replace(" ",""))
    # jieba.cut()按照词性切分“我爱中国”切分成我爱中国
    test["X_split"]=test["X"].apply(lambda x:" ".join(jieba.cut(x)))
    return test
 
testData=process_test("Test_DataSet.csv")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7356 entries, 0 to 7355
Data columns (total 4 columns):
id         7356 non-null object
title      7356 non-null object
content    7288 non-null object
X          7356 non-null object
dtypes: object(4)
memory usage: 230.0+ KB
None


In [6]:
train_x = dataDropNa.X_split.values # 训练集文本分词数据
train_y = dataDropNa.label.values # 训练集标签
test_x = testData.X_split.values #测试集文本分词数据

In [88]:
print(dataDropNa["X_split"])

0       问责 领导 ( 上 黄镇 党委书记 张涛 ， 宣国 才 真能 一手遮天 吗 ？ ) 这 几天...
1       江歌 事件 : 教会 孩子 ， 善良 的 同时 更要 懂得 保护 自己 ! 过去 一年 的 ...
2       绝味 鸭 脖 广告 " 开 黄腔 " 引 众怒 " 双 11 " 这么 拼值 吗 ? “ 双...
3       央视 曝光 ! 如东 一 医药企业 将 槽罐车 改成 垃圾车 ， 夜间 偷排 高浓度 废水 ...
4       恶劣 至极 ， 央视 都 曝光 了 ! 南通 如东 一 医药企业 将 槽罐车 改成 洒水车 ...
5       央视 曝光 ! 南通 一 医药企业 夜间 偷排 高浓度 废水 丢脸 ! 昨晚 央视 一套 晚...
6       粉丝 爆料 : 五洲 国际 无锡 项目 涉嫌 诈骗 、 非法 集资 随着 金融街 1 号 （...
7       年内 约 10 起 锂电 重组 失败 资本 对 高 估值 收购 说 “ 不 ” 摘要 来自 ...
8       男子 梦想 一夜 暴富 持 水泥块 砸机 一分钱 都 没取 到 近日 ， 江苏 扬州 ， 谢...
9       北京 多家 法院 供暖 纠纷 : 案件 主体 为 供暖费 追缴 山海 网 北京 的 这个 冬...
10      手机号 、 、 开头 的 注意 ! 看 完 吓 得 冷汗 都 出来 了 搜 搜狐 原 标题 ...
11      网红 “ 土坯房 书记 ” 落马 的 背后 原 标题 : 网红 “ 土坯房 书记 ” 落马 ...
12      曾经 的 品质 烤肉 ， 两年 你 就 这么 难吃 了 三鼎 甲 ， 曾经 在 哈尔滨 烤肉...
13      土俄 军火交易 再起 波折 ， 鱼熊 无法 兼顾 ， 土 或 成美 俄 交锋 牺牲品 搜狐 ...
14      我 在 一个 叫 “ 闲鱼 ” 的 网上交易 平台 购买 了 一个 代付款 服务 ， 就是 ...
15      标题 : 不 签字 ， 不 同意 ， 不 反抗 ， 让 某些 人扮 普天同庆 的 样子 好 ...
16      这伙 是 他 自已 说 的 教育 废品 ， 文 不能 作文 。 一篇 文章 写 着 写 着 ...
17      车震 视频 

In [93]:
# train = np.array(dataDropNa)
# train_list=train.tolist()#list
# print(train_list)
# print(type(train_list))
# 转换成列表

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.



<class 'list'>


In [72]:
# def strs_filter(lines):
# #    with codecs.open(file,"r","utf8") as f,codecs.open("result.txt","a+","utf8") as c:
#  #       lines=f.readlines()
#         #for line in lines:
#             # line=line.decode('utf8')
#             re_html=re.compile('<[^>]+>'.decode('utf8'))  # 从'<'开始匹配，不是'>'的字符都跳过，直到'>'
#             re_punc=re.compile('[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*“”《》：（）]+'.decode('utf8'))#去除标点符号
#             re_digits_letter=re.compile('\w+'.decode('utf8'))#去除数字及字母
#             line=re_html.sub('',lines)
#             line=re_punc.sub("",lines)
#             line=re_digits_letter.sub("",lines)
#             c.write(lines)
# strs_filter(a)
# pre = re.compile('>(.*?)<') 
# s1 = ''.join(pre.findall(a))
# print(s1)   # '随笔文章日记评论链接相册文件设置选项'
def clean(contents):
    for content in contents:
        
        s1 = re.sub("[a-zA-Z0-9]","", content).replace(r'/','')
        s = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）”“:=图片)?;>：-]+", "",s1) 
        

print(s)


这几天看了有人举报施某某的贴子经与举报人联系证实是宣某当天中午请举报人和枪手喝酒后晚上才发的贴子本人不去讨论前二天的举报相信总归会有说法的今天一看施全军年月日实名举报上黄镇宣国才的贴子仍被锁定禁止评论已经正好一整年了施全军实名举报天后上黄镇党委政府回复如下一年的贴子再次被网友顶起来后才发现施某几天前回复网友的处理结果竟如下现责问张涛书记宣国才被举报这么多问题什么时候有答复宣国才被举报后为什么被立刻免了村书记职务为什么又被安排到城管队“吃空响”自己却天天在我们水泥厂上班赚黑钱这几个月水泥每吨近元纯利润还供不应求宣国才还清上黄政府担保借给宣国才代付振东厂工资社保的钱了吗据了解宣国才占他人企业经营又欠税万元欠社保万元应该还欠了职工工资几十万上黄政府打算替宣国才担保还是归还我们厂合法会计和老板被判刑四到六年现在服刑厂子给宣国才强占宣国才每天赚多万净利润却对外宣称天天亏本等咱老板刑满回厂宣国才给咱厂“天天亏”可能要“亏”的几千万元甚至几个亿张涛书记您承担还是上黄政府承担当初可是您亲自把厂交给宣国才生产的希望徐市长看到本贴后能像批示批示违建等民生问题一样关注一下我们水泥厂的将来也请徐市长抽日理万机之空亲自约谈一下当事人特别是那位施站长千万不能听取一面之辞


In [24]:
print(text)

这几天看了有人举报施某某的贴子，经与举报人联系证实，是宣某当天中午请举报人和枪手喝酒后，晚上才发的贴子！ 本人不去讨论前二天的举报，相信总归会有说法的！ 今天一看施全军2017年1月2日实名举报上黄镇宣国才的贴子（仍被锁定禁止评论）已经正好一整年了 =750) window.open('http://img.jsly001.com/attachment/mon_1801/4_291085_c796a6a86e17121.jpg?123');" onload="if(this.offsetwidth>'750')this.width='750';" src="http://img.jsly001.com/attachment/mon_1801/4_291085_c796a6a86e17121.jpg?123" style="max-width:750px;"/>图片:/home/alidata/www/data/tmp/qfupload/4_291085_1514981471478952.jpg 施全军实名举报50天后，上黄镇党委政府回复如下图： =750) window.open('http://img.jsly001.com/attachment/mon_1801/4_291085_a9b11b7ea2b1ce9.jpg?90');" onload="if(this.offsetwidth>'750')this.width='750';" src="http://img.jsly001.com/attachment/mon_1801/4_291085_a9b11b7ea2b1ce9.jpg?90" style="max-width:750px;"/>图片:/home/alidata/www/data/tmp/qfupload/4_291085_1514981472631668.jpg =750) window.open('http://img.jsly001.com/attachment/mon_1801/4_291085_9cde9b3943fe20c.jpg?75');" onload="if(this.offsetwidth>'750')this.width='750';" src="http://img.jsly001.com/attachment/mon

In [21]:
def content_to_wordlist(content, remove_stopwords=False):
    # 1. Remove HTML 删除html符号
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters 删除非字母符号，后续可以考虑不删除数字
    # review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. 把所有单词转换成小写然后将文本分割成单词
    # words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default) 有选择的删除stopwords
#     if remove_stopwords:
#         stops = set(stopwords.words("english"))
#         words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return content

In [1]:
import gensim
model = gensim.models.Word2Vec(X, size=100)

NameError: name 'X' is not defined

X是经分词后的文本构成的list，也就是tokens的列表的列表.

    注意，Word2Vec还有3个值得关注的参数，iter是模型训练时迭代的次数，假
    如参与训练的文本量较少，就需要把这个参数调大一些；sg是模型训练算法的
    类别，1 代表 skip-gram，;0代表 CBOW;window控制窗口，它指当前词和预测
    词之间的最大距离，如果设得较小，那么模型学习到的是词汇间的功能性特征
    （词性相异），如果设置得较大，会学习到词汇之间的相似性特征（词性相
    同）的大小，假如语料够多，笔者一般会设置得大一些，8~10。